# Notes: this Java notebook requires Ganymede 
* Ganymede (JShell in Jupyter): [Installation and documentation](https://github.com/allen-ball/ganymede)
* We need additional libraries for lucene

In [92]:
%%pom
dependencies:
- org.apache.lucene:lucene-core:LATEST
- org.apache.lucene:lucene-analyzers-common:LATEST
- org.apache.lucene:lucene-queryparser:LATEST

#### Common imports (java)

In [194]:
import java.io.BufferedReader;
import java.io.FileReader;
import java.io.IOException;
import java.io.Reader;
import java.io.StringReader;

import java.util.Arrays;
import java.util.ArrayList;
import java.util.HashMap;
import java.util.Map;
import java.util.regex.Pattern;

#### Common imports (lucene)

In [ ]:
import org.apache.lucene.analysis.Analyzer;
import org.apache.lucene.analysis.TokenStream;  
import org.apache.lucene.analysis.Tokenizer;
import org.apache.lucene.analysis.core.LowerCaseFilter;
import org.apache.lucene.analysis.en.EnglishAnalyzer;
import org.apache.lucene.analysis.en.EnglishPossessiveFilter;
import org.apache.lucene.analysis.en.KStemFilter;
import org.apache.lucene.analysis.standard.StandardAnalyzer;
import org.apache.lucene.analysis.standard.StandardFilter;
import org.apache.lucene.analysis.standard.StandardTokenizer;
import org.apache.lucene.analysis.tokenattributes.CharTermAttribute;
import org.apache.lucene.analysis.tokenattributes.TypeAttribute;
import org.apache.lucene.analysis.util.CharArraySet;

import org.apache.lucene.util.Version;

## Let's read in the data collection

In [195]:
ArrayList<Map<String, String>> read_collection(String name) {
    ArrayList<Map<String, String>> docs = new ArrayList<Map<String, String>>();
    String splitter = ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)";
    
    try{
        BufferedReader reader = new BufferedReader(new FileReader(name));
        String line, keys[] = reader.readLine().split(splitter);

        while ((line = reader.readLine()) != null) {
            String[] values = line.split(splitter);
            Map<String, String> dataMap = new HashMap<>();

            for (int i = 0; i < keys.length; i++) {
                // dataMap.put(keys[i], values[i]);
                switch(keys[i]){
                    case "Series_Title":
                        dataMap.put("title", values[i]);
                        break;
                    case "Released_Year":
                        dataMap.put("year", values[i]);
                        break;
                    case "Runtime":
                        dataMap.put("runtime", values[i].replace(" min", ""));
                        break;
                    case "Genre":
                        dataMap.put("genre", values[i].replace(",",""));
                        break;
                    case "IMDB_Rating":
                        dataMap.put("rating", values[i]);
                        break;
                    case "Overview":
                        dataMap.put("summary", values[i]);
                        break;
                    case "Star1":
                        dataMap.put("actors", values[i]);
                        break;
                    case "Star2":
                    case "Star3":
                    case "Star4":
                        dataMap.put("actors", dataMap.get("actors") + " " + values[i]);
                        break;
                }
            }
            docs.add(dataMap);
        }
    } catch(IOException e) {
        e.printStackTrace();
    }
    System.out.println("Read " + docs.size() + " documents from " + name);
    return docs;
}

var collection = read_collection("datasets/imdb_top_1000.csv");

Read 1000 documents from datasets/imdb_top_1000.csv


## Let's start with the analyzer of Lucene

In [193]:


void print_tokens(Analyzer analyzer, String text) {
    try {
        TokenStream ts = analyzer.tokenStream("text", new StringReader(text));
        CharTermAttribute termAtt = ts.addAttribute(CharTermAttribute.class);

        try {
            ts.reset();
            // Print all tokens until stream is exhausted
            while (ts.incrementToken()) 
                System.out.print(termAtt.toString() + " ");
            ts.end();
            System.out.println();
        } finally {
            ts.close();
        }
    } catch(IOException e) {
        e.printStackTrace();
    }    
}

class MyAnalyzer extends Analyzer {
    MyAnalyzer(Version matchVersion) {
    }
    @Override
    protected TokenStreamComponents createComponents(String fieldName, Reader reader) {
        Version matchVersion = Version.LUCENE_CURRENT;
        final Tokenizer source = new StandardTokenizer(matchVersion, reader);
        TokenStream result = new StandardFilter(matchVersion, source);
        result = new EnglishPossessiveFilter(matchVersion, result);
        // result = new LowerCaseFilter(matchVersion, result);
        result = new KStemFilter(result);
        return new TokenStreamComponents(source, result);
    }
}

var text = "I think text's values' color goes here; WHAT happens with it? do we see IT again; I went there to be gone with houses";
var stopWords = new CharArraySet(Version.LUCENE_CURRENT, Arrays.asList("i", "do"), false);

System.out.println("             text: "+ text);
System.out.println();

// standard analyzer
System.out.print("         standard: ");
print_tokens(new StandardAnalyzer(Version.LUCENE_CURRENT), text);

// english analyzer (with porter stemmer)
System.out.print("          english: ");
print_tokens(new EnglishAnalyzer(Version.LUCENE_CURRENT), text);

// english analyzer (with porter stemmer) and new set of stopwords
System.out.print("english/stopwords: ");
print_tokens(new EnglishAnalyzer(Version.LUCENE_CURRENT, stopWords), text);

// a custom analyzer, no lower case and kstemmer
System.out.print("      my analyzer: ");
print_tokens(new MyAnalyzer(Version.LUCENE_CURRENT), text);

EnglishAnalyzer.getDefaultStopSet()

[but, be, with, such, then, for, no, will, not, are, and, their, if, this, on, into, a, or, there, in, that, they, was, is, it, an, the, as, at, these, by, to, of]

             text: I think text's values' color goes here; WHAT happens with it? do we see IT again; I went there to be gone with houses

         standard: i think text's values color goes here what happens do we see again i went gone houses 
          english: i think text valu color goe here what happen do we see again i went gone hous 
english/stopwords: think text valu color goe here what happen with it we see it again went there to be gone with hous 
      my analyzer: I think text value color go here WHAT happen with it do we see IT again I went there to be gone with house 


## Building an index (in memory)

In [5]:
import org.knowm.xchart.XYChart;
import org.knowm.xchart.XYChartBuilder;

var xchart = new XYChartBuilder().title("Trig").build();

xchart.addSeries("sin", x, sinx);
xchart.addSeries("cos", x, cosx);

print(xchart)

REJECTED ERRONEOUS


print(xchart)
cannot find symbol
  symbol:   method print(org.knowm.xchart.XYChart)
  location: class 
